# How to compute a Neural Adjoint Map

In [2]:
import geomstats.backend as gs

from geomfum.dataset import NotebooksDataset
from geomfum.refine import NeuralZoomOut
from geomfum.shape import TriangleMesh

In [3]:
dataset = NotebooksDataset()

mesh_a = TriangleMesh.from_file(dataset.get_filename("cat-00"))
mesh_b = TriangleMesh.from_file(dataset.get_filename("lion-00"))

mesh_a.n_vertices, mesh_b.n_vertices

INFO: Data has already been downloaded... using cached file ('C:\Users\giuli\.geomfum\data\cat-00.off').
INFO: Data has already been downloaded... using cached file ('C:\Users\giuli\.geomfum\data\lion-00.off').


(7207, 5000)

In [4]:
mesh_a.laplacian.find_spectrum(spectrum_size=50, set_as_basis=True)
mesh_b.laplacian.find_spectrum(spectrum_size=50, set_as_basis=True)

mesh_a.basis.use_k = 10
mesh_b.basis.use_k = 10

In [5]:
from geomfum.convert import NearestNeighbors

finder = NearestNeighbors(n_neighbors=1)
finder.fit(mesh_b.vertices)
p2p = finder.kneighbors(mesh_a.vertices)[1].flatten()

In [6]:
from geomfum.convert import NamFromP2pConverter


nam_converter = NamFromP2pConverter(device="cuda")

nam = nam_converter(p2p, mesh_a.basis, mesh_b.basis)


In [7]:
print(nam)

NeuralAdjointMap(
  (linear_branch): Linear(in_features=10, out_features=10, bias=False)
  (nonlinear_branch): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=10, out_features=128, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=128, out_features=10, bias=True)
    )
  )
)


In [8]:
from geomfum.convert import P2pFromNamConverter


p2p_from_nam = P2pFromNamConverter()

p2p_from_nam(nam, mesh_a.basis, mesh_b.basis)

array([  11,   13,   88, ..., 4831, 4851,   11], shape=(7207,))

In [ ]:
from geomfum.refine import NeuralZoomOut

nzo = NeuralZoomOut(nit=10, step=2, device="cuda")


nam_ref = nzo(nam, mesh_a.basis, mesh_b.basis)